In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp loss_functions

# Loss Functions

> Functions to find the loss (or error) given a set of observations and point forecasts.

The loss functions in this module are commonly used to complete a time series analysis, with the following steps:

1. Run an `analysis`, returning an array of forecast samples
2. Use a function from `point_forecast` to find the forecast mean, median, or $(-1)-$median
3. Evaluate the point forecast error using a loss function from this module

In [ ]:
#hide
#exporti
import numpy as np

In [ ]:
#export
def MSE(y, f):
    """
    Mean squared error (MSE).
    """

    y = np.ravel(y)
    f = np.ravel(f)
    return np.mean((y - f)**2)

The mean squared error loss is:

$$L = \frac{1}{n} \sum_{i=1:n} (y_i - f_i)^2$$

The mean is the optimal point forecast for minimizing the MSE. An example below demonstrates how to use the function:

In [ ]:
from pybats.shared import load_us_inflation
from pybats.analysis import analysis
from pybats.point_forecast import mean, median, m_one_median
import pandas as pd

data = load_us_inflation()

forecast_start = '2000-Q1'
forecast_end = '2013-Q4'
Y = data.Inflation.values[1:]

mod, samples = analysis(Y = Y, X=None, family="normal",
                        k = 4, prior_length = 12,
                        forecast_start = forecast_start, forecast_end = forecast_end,
                        dates=data.Date,
                        ntrend = 2, deltrend=.99,
                        nsamps = 5000)

# Use the mean, because it is optimal for minimizing the MSE
forecast = mean(samples)

for h in range(4):
    start = data[data.Date == forecast_start].index[0] + h
    end = data[data.Date == forecast_end].index[0] + h + 1

    print(str(h+1) + '-Step Ahead MSE: ' + str(MSE(Y[start:end], forecast[:,h-1]).round(2)))

beginning forecasting
1-Step Ahead MSE: 1.08
2-Step Ahead MSE: 1.01
3-Step Ahead MSE: 1.08
4-Step Ahead MSE: 1.13


In [ ]:
#export
def MAD(y, f):
    """
    Mean absolute deviation (MAD).
    """

    y = np.ravel(y)
    f = np.ravel(f)
    return np.mean(np.abs(y-f))

The mean absolute deviation is:

$$L = \frac{1}{n} \sum_{i=1:n} |y_i - f_i|$$

The median is the optimal point forecast for minimizing the MAD. An example below demonstrates how to use the function:

In [ ]:
# Use the median, because it is optimal for minimizing the MAD
forecast = median(samples)

for h in range(4):
    start = data[data.Date == forecast_start].index[0] + h
    end = data[data.Date == forecast_end].index[0] + h + 1

    print(str(h+1) + '-Step Ahead MAD: ' + str(MAD(Y[start:end], forecast[:,h-1]).round(2)))

1-Step Ahead MAD: 0.85
2-Step Ahead MAD: 0.8
3-Step Ahead MAD: 0.82
4-Step Ahead MAD: 0.84


In [ ]:
#export
def MAPE(y, f):
    """
    Mean absolute percent error (MAPE).
    """
    y = np.ravel(y)
    f = np.ravel(f)
    return 100*np.mean(np.abs((y - f)) / y)

The mean absolute percent error is:

$$L = \frac{1}{n} \sum_{i=1:n} \frac{|y_i - f_i|}{y_i}$$

The $(-1)-$median is the optimal point forecast for minimizing the MAPE. An example below demonstrates how to use the function:

In [ ]:
# Use the (-1)-median, because it is optimal for minimizing the MAPE
forecast = m_one_median(samples)

for h in range(4):
    start = data[data.Date == forecast_start].index[0] + h
    end = data[data.Date == forecast_end].index[0] + h + 1

    print(str(h+1) + '-Step Ahead MAPE: ' + str(MAPE(Y[start:end], forecast[:,h-1]).round(2)) + "%")

1-Step Ahead MAPE: 73.37%
2-Step Ahead MAPE: 90.54%
3-Step Ahead MAPE: 66.98%
4-Step Ahead MAPE: 80.82%


In [ ]:
#exporti
def WAPE(y, f):
    """
    Weighted Absolute Percent Error (WAPE).

    .. math:: WAPE(y, f) =  \\frac{\sum_{i=1:n} |y_i-f_i|}{\sum_{i=1:n} y_i}

    The weighted absolute percent error solves the issues of division by 0 in the MAPE.

    The optimal point forecase to minimize the WAPE is the joint (-1)-median.

    .. code::

        k = 1
        WAPE(y[forecast_start + k - 1:forecast_end + k], joint_m_one_median(samples))

    :param y: Observation vector
    :param f: Point forecast vector
    :return: Weighted absolute percent error (WAPE)
    """

    y = np.ravel(y)
    f = np.ravel(f)
    return 100*np.sum(np.abs(y-f)) / np.sum(y)

In [ ]:
#exporti
def WAFE(y, f):
    """
    Weighted Absolute Forecast Error (WAFE).

    .. math:: WAFE(y, f) =  \\frac{\sum_{i=1:n} |y_i-f_i|}{\sum_{i=1:n} \\frac{1}{2}(y_i + f_i)}

    Similar to WAPE, the weighted absolute forecast error solves the issues of division by 0 in the MAPE.
    It  balances between the observations (:math:`y_i`) and forecasts (:math:`f_i`) in the denominator.

    .. code::

        k = 1
        WAFE(y[forecast_start + k - 1:forecast_end + k], median(samples))

    :param y: Observation vector
    :param f: Point forecast vector
    :return: Weighted absolute forecast error (WAFE)
    """

    y = np.ravel(y)
    f = np.ravel(f)
    return 100*np.sum(np.abs(y-f)) / ((np.sum(y) + np.sum(f))/2)

In [ ]:
#exporti
def ZAPE(y, f):
    """
    Zero-Adjusted Absolute Percent Error (ZAPE).

    .. math:: ZAPE(y, f) = \\frac{1}{n} \sum_{i=1:n} I(y_i = 0) * f_i + I(y_i = 1) * |y_i-f_i| / y_i

    The zero-adjusted absolute percent error is an similar to absolute percent error (APE), but sets the loss equal to :math:`f_i` when :math:`y_i=0`, to avoid division by 0.

    Finding the optimal point forecast for ZAPE requires a simple numerical optimization, and lies between the median and (-1)-median.

    .. code::

        k = 1
        ZAPE(y[forecast_start + k - 1:forecast_end + k], median(samples))

    :param y: Observation vector
    :param f: Point forecast vector
    :return: The mean Zero-Adjusted absolute percent error (ZAPE)
    """

    y = np.ravel(y)
    f = np.ravel(f)
    nonzeros = y.nonzero()[0]
    n = len(y)
    loss = np.copy(f)
    loss[nonzeros] = np.abs(y[nonzeros] - f[nonzeros]) / y[nonzeros]
    return 100*np.mean(loss)

In [ ]:
#exporti
def scaledMSE(y, f, ymean = None):
    y = np.ravel(y)
    f = np.ravel(f)
    if ymean is None:
        # First check if the 'y' vector is longer than f
        ny = len(y)
        nf = len(f)
        ymean = np.cumsum(y) / np.arange(1, ny+1)
        # Assume that the forecasts and y vector terminate at the same point
        y = y[-nf:]
        ymean = ymean[-nf:]
    return np.mean(((y.reshape(-1) - f.reshape(-1)) ** 2 / (ymean.reshape(-1) ** 2)))

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_dglm.ipynb.
Converted 01_update.ipynb.
Converted 02_forecast.ipynb.
Converted 03_define_models.ipynb.
Converted 04_seasonal.ipynb.
Converted 05_analysis.ipynb.
Converted 06_conjugates.ipynb.
Converted 07_point_forecast.ipynb.
Converted 08_loss_functions.ipynb.
Converted 09_plot.ipynb.
Converted 10_shared.ipynb.
Converted 11_dcmm.ipynb.
Converted 12_dbcm.ipynb.
Converted 13_latent_factor.ipynb.
Converted 14_latent_factor_fxns.ipynb.
Converted index.ipynb.
